<a href="https://colab.research.google.com/github/dnkhiem/Crypto-prediction/blob/master/stock_predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow==2.0.0
!pip install python-binance
!wget --no-check-certificate \
    https://raw.githubusercontent.com/dnkhiem/Crypto-prediction/master/original_data/data_numpy_1h.csv?token=AEFIVV7W4WNC7F7MOMS7TRK57QRBK \
    -O tmp/data_numpy_1h.csv

tmp/data_numpy_1h.csv: No such file or directory


In [0]:
from binance.client import Client
import os

import tensorflow as tf
import numpy as np
import pandas as pd    
import matplotlib.pyplot as plt
from datetime import datetime

api_key = '9dCDlPKBoql51ImypE4NDCRwq68fFlav2Ttf3OfRJliZb6jQAZze7hWY20ez1K73'
api_secret = '2xLoIawNjTYL2rImE7RemR3GleQFaqz6WKhrYsXlqV5cFfRhuzLzHAlU18DKlLHu'



In [0]:
def get_data_from_binance(client, symbol, time_candle, time_begin = "1 Jan, 2017"):
    time_dict = {
        '15m':Client.KLINE_INTERVAL_15MINUTE,
        '30m':Client.KLINE_INTERVAL_30MINUTE,
        '1h':Client.KLINE_INTERVAL_1HOUR,
        '2h':Client.KLINE_INTERVAL_2HOUR,
        '4h':Client.KLINE_INTERVAL_4HOUR,
        '6h':Client.KLINE_INTERVAL_6HOUR,
        '8h':Client.KLINE_INTERVAL_8HOUR,
        '12h':Client.KLINE_INTERVAL_12HOUR,
        '1d':Client.KLINE_INTERVAL_1DAY,
        '3d':Client.KLINE_INTERVAL_3DAY,
        }
    klines = client.get_historical_klines(symbol, time_dict[time_candle], time_begin, "now UTC")   
    return klines

def get_data(client, symbol, cols, time_scale, update_data = True):
    filename = '/tmp/data_numpy_' + str(time_scale) + '.csv'
    data_cols_label = ['Open time', 'Open', 'High', 'Low', 'Close', 'Volume', 
                           'Close Time', 'Quote asset volume', 'Number of trades',
                           ' Taker buy base asset volume', 'Taker buy quote asset volume',
                           'Ignore']
    if os.path.exists(filename):
        print('file exists')
        df1 = pd.read_csv(filename) 
        if update_data == False:
            return df1.astype(float)
        timestamp = df1.iloc[-1, 0]
        dt_object = datetime.fromtimestamp(timestamp*1e-3)
        time_begin = dt_object.strftime('%d %b, %Y')        
        datas = get_data_from_binance(client, symbol, time_candle = time_scale, time_begin = time_begin)
        df2 = pd.DataFrame(datas)
        df2.set_axis(data_cols_label, axis='columns', inplace=True)
        df2 = df2.iloc[:,cols]
        df2.index.name = 'Index'
        #combine 2 dataframe
        time_str = datetime.strptime(time_begin, '%d %b, %Y')
        time_stamp_begin = int(datetime.timestamp(time_str) *1000)
        print(time_stamp_begin)
        print(df1[df1.loc[:,'Open time'] == time_stamp_begin])
        index_combine = df1[df1.loc[:,'Open time'] == time_stamp_begin].index[0]
        df1 =df1.iloc[:index_combine]
        df3 = pd.concat([df1, df2], ignore_index=True)
        df3.to_csv(filename, index=False)
        return df3.astype(float)
    else:        
        datas = get_data_from_binance(client, symbol, time_candle = time_scale)
#    data_numpy = np.asarray(datas, np.float32)    
        df = pd.DataFrame(datas)
        
        df.set_axis(data_cols_label, axis='columns', inplace=True)
        df.index.name = 'Index'
        df2 = df.iloc[:,cols]
        df2.to_csv(filename, index=False)
        return df2.astype(float)


In [4]:
client = Client(api_key, api_secret)
symbol = 'BTCUSDT'
cols = [0, 1, 2, 3, 4, 5]
cols_for_LSTMs = [1, 2, 3, 4, 5, 6]
data1 = get_data(client, symbol, cols, time_scale = '1h', update_data = False)
data1['Avg'] = (data1['Open'] + data1['Close'])/2


file exists


In [5]:
print(data1.shape)

(20422, 7)


In [0]:
def windowed_dataset(data_in, windows_slide = 30):
    data_out = np.zeros((data_in.shape[1], data_in.shape[0] - windows_slide+1, windows_slide))
    print(data_in.shape)
    print(data_out.shape)
    for i in range(data_in.shape[0] - windows_slide+1):  
        data_out[:,i,:] = data_in.T[:,i:i+windows_slide]
    return data_out


In [0]:
def get_data_train(data, cols, windows_slide = 30, train_ratio = 0.9):
    data_in = np.array(data.iloc[:,cols])
    data_for_LSTMs = windowed_dataset(data_in, windows_slide)
    
    n_data = data_for_LSTMs.shape[1]
    print(data_for_LSTMs.shape)
    split_time = int(n_data * train_ratio)
    
    X_train = data_for_LSTMs[:,:split_time,:]
    y_train = data_in[windows_slide:windows_slide + split_time,-1]
    
    X_valid = data_for_LSTMs[:,split_time:-1,:]
    y_valid = data_in[windows_slide + split_time:,-1]
    
    X_train = np.transpose(X_train, (1, 2, 0))
    X_valid = np.transpose(X_valid, (1, 2, 0))
    
    return X_train, y_train, X_valid, y_valid

In [0]:
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('acc')>0.99 and logs.get('val_acc')>0.95):
      print("\nReached 99% accuracy so cancelling training!")
      self.model.stop_training = True

In [0]:
def separate_block(input_tensor, n_LSTMs = 16, n_Dense = 16):
  x = tf.keras.layers.Lambda(lambda x: tf.expand_dims(x, axis=-1))(input_tensor)
  x = tf.keras.layers.Conv1D(n_LSTMs, 5, activation = 'relu') (x) # test with Conv1D
  x = tf.keras.layers.BatchNormalization()(x)
# x = tf.keras.layers.Dropout(0.2)(x)
  x = tf.keras.layers.LSTM(n_LSTMs, return_sequences=True)(x)
  x = tf.keras.layers.BatchNormalization()(x)
  x = tf.keras.layers.LSTM(n_LSTMs)(x)
#  x = tf.keras.layers.BatchNormalization()(x)
#  x = tf.keras.layers.Dense(n_Dense, activation="relu")(x)
  return x

In [0]:
def build_model_find_lr(X_train, y_train, X_valid, y_valid):
    tf.keras.backend.clear_session()
    tf.random.set_seed(51)
    np.random.seed(51)

    batch_size = 32
    epochs = 50
    callback = myCallback()
    input_data = tf.keras.layers.Input(shape = [None, 6])


    x = tf.keras.layers.Conv1D(64, 5, activation = 'relu') (input_data) # test with Conv1D
    x = tf.keras.layers.LSTM(64, return_sequences=True)(x)
    x = tf.keras.layers.LSTM(64)(x)
#    x = tf.keras.layers.BatchNormalization()(x)    
    x = tf.keras.layers.Dense(64, activation="relu")(x)
#    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Dense(1)(x)

    model = tf.keras.models.Model(input_data, x)
    model.summary()

    lr_schedule = tf.keras.callbacks.LearningRateScheduler(
    lambda epoch: 1e-6 * 10**(epoch / 20))
    optimizer = tf.keras.optimizers.SGD(lr=1e-6, momentum=0.9)
    model.compile(loss=tf.keras.losses.Huber(),
              optimizer=optimizer,
              metrics=["mae"])
    history = model.fit(X_train, y_train, epochs=80, callbacks=[lr_schedule])
    
    plt.semilogx(history.history["lr"], history.history["loss"])
    plt.axis([1e-8, 1e-1, 0, 60])
    
    return history, model

In [0]:

X_train, y_train, X_valid, y_valid = get_data_train(data1, cols_for_LSTMs, train_ratio = 0.95)
history, model = build_model_find_lr(X_train, y_train, X_valid, y_valid)


(20422, 6)
(6, 20393, 30)
(6, 20393, 30)
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None, 6)]         0         
_________________________________________________________________
conv1d (Conv1D)              (None, None, 64)          1984      
_________________________________________________________________
lstm (LSTM)                  (None, None, 64)          33024     
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                33024     
_________________________________________________________________
dense (Dense)                (None, 64)                4160      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total params: 72,257
Trainable params: 72,257
Non-trainable params: 0
________________